# Data sourcing

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib
%matplotlib inline
import numpy as np
import pandas as pd

**Goal 1:** get song data from a Spotify dump CSV.

## 1️⃣ CSV

In [4]:
!tree

.
├── __pycache__
│   ├── music.cpython-310.pyc
│   └── music.cpython-38.pyc
├── bad-import-example
│   ├── __pycache__
│   │   ├── bad_import_lib.cpython-38.pyc
│   │   └── good_import_lib.cpython-38.pyc
│   ├── bad_import_lib.py
│   ├── good_import_lib.py
│   └── main.py
├── data
│   ├── music.sqlite
│   └── spotify_2017.csv
├── data-sourcing-ref.ipynb
├── data-sourcing.ipynb
├── music.py
├── recap
│   ├── README.md
│   ├── Recap.ipynb
│   └── books.csv
└── recap-ref
    ├── README.md
    ├── Recap.ipynb
    └── books.csv

7 directories, 18 files


In [5]:
tracks_df = pd.read_csv('data/spotify_2017.csv')

In [6]:
tracks_df[['name', 'artists']].head()

,name,artists
0,Shape of You,Ed Sheeran
1,Despacito - Remix,Luis Fonsi
2,Despacito (Featuring Daddy Yankee),Luis Fonsi
3,Something Just Like This,The Chainsmokers
4,I'm the One,DJ Khaled


In [7]:
tracks_df.info
tracks_df.shape

(100, 16)

**Goal 2:** fetch lyrics and add them as new column of our dataframe track_df
Where could we get that data from? 

## 2️⃣ API

In [8]:
from music import fetch_lyrics

In [9]:
%%time
# for index, row in tracks_df.iterrows():
#     tracks_df.loc[index, 'lyrics'] = fetch_lyrics(row['artists'], row['name'])

tracks_df['lyrics'] = tracks_df.apply(lambda x : fetch_lyrics(x['artists'], x['name']), axis=1)

# NOTE regarding axis=1:
# axis {0 or ‘index’, 1 or ‘columns’}, default 0
# Axis along which the function is applied:
# 0 or ‘index’: apply function to each column.
# 1 or ‘columns’: apply function to each row.

CPU times: user 2.55 s, sys: 150 ms, total: 2.7 s
Wall time: 1min 53s


In [10]:
tracks_df.head(5)

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1.0,-3.183,0.0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,233713.0,4.0,The club isn't the best place to find a lover\...
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2.0,-4.328,1.0,0.1200,0.2290,0.000000,0.0924,0.813,88.931,228827.0,4.0,No lyrics
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.846,177.833,228200.0,4.0,No lyrics
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11.0,-6.769,0.0,0.0317,0.0498,0.000014,0.1640,0.446,103.019,247160.0,4.0,I've been reading books of old\nThe legends an...
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7.0,-4.284,1.0,0.0367,0.0552,0.000000,0.1670,0.811,80.924,288600.0,4.0,[DJ Khaled:]\nWe The Best Music\nAnother One!\...


**Goal 3:** add the amount of LastFM listener per song from a LastFM DB dump

## 3️⃣ SQL

In [11]:
query = """
    SELECT ai.artist_mb, p.listeners_lastfm
    FROM artist_info ai 
    JOIN popularity p ON ai.mbid = p.mbid
    WHERE p.listeners_lastfm >= ?
"""

In [12]:
limit = "500000"

In [13]:
import sqlite3

In [14]:
conn = sqlite3.connect('data/music.sqlite')

In [15]:
# c = conn.cursor()
# c.execute(query)
# data = c.fetchall()
# list(data)[:2]

In [16]:
listen_df = pd.read_sql(query, conn, params=(limit,))
listen_df

,artist_mb,listeners_lastfm
0,Coldplay,5381567.0
1,Radiohead,4732528.0
2,Red Hot Chili Peppers,4620835.0
3,Rihanna,4558193.0
4,Eminem,4517997.0
...,...,...
1847,Future Islands,500380.0
1848,Bobby Helms,500364.0
1849,Inner Circle,500335.0
1850,War,500332.0


In [17]:
tracks_df = tracks_df.merge(
    listen_df,
    left_on= 'artists',
    right_on='artist_mb',
    how='left'
)

In [18]:
tracks_df

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,artist_mb,listeners_lastfm
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1.0,-3.183,0.0,0.0802,0.5810,0.000000,0.0931,0.9310,95.977,233713.0,4.0,The club isn't the best place to find a lover\...,Ed Sheeran,1620379.0
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2.0,-4.328,1.0,0.1200,0.2290,0.000000,0.0924,0.8130,88.931,228827.0,4.0,No lyrics,NaN,NaN
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.8460,177.833,228200.0,4.0,No lyrics,NaN,NaN
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11.0,-6.769,0.0,0.0317,0.0498,0.000014,0.1640,0.4460,103.019,247160.0,4.0,I've been reading books of old\nThe legends an...,The Chainsmokers,794664.0
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7.0,-4.284,1.0,0.0367,0.0552,0.000000,0.1670,0.8110,80.924,288600.0,4.0,[DJ Khaled:]\nWe The Best Music\nAnother One!\...,DJ Khaled,1160535.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1PSBzsahR2AKwLJgx8ehB,Bad Things (with Camila Cabello),Machine Gun Kelly,0.675,0.690,2.0,-4.761,1.0,0.1320,0.2100,0.000000,0.2870,0.2720,137.817,239293.0,4.0,I can't stop thinkin'\nThinking 'bout sinkin'\...,NaN,NaN
96,0QsvXIfqM0zZoerQfsI9l,Don't Let Me Down,The Chainsmokers,0.542,0.859,11.0,-5.651,1.0,0.1970,0.1600,0.004660,0.1370,0.4030,159.797,208053.0,4.0,No lyrics,The Chainsmokers,794664.0
97,7mldq42yDuxiUNn08nvzH,Body Like A Back Road,Sam Hunt,0.731,0.469,5.0,-7.226,1.0,0.0326,0.4630,0.000001,0.1030,0.6310,98.963,165387.0,4.0,Folks said that you found someone new\nTo do t...,NaN,NaN
98,7i2DJ88J7jQ8K7zqFX2fW,Now Or Never,Halsey,0.658,0.588,6.0,-4.902,0.0,0.0367,0.1050,0.000001,0.1250,0.4340,110.075,214802.0,4.0,No lyrics,Halsey,564334.0


**Goal 4:** add another track popularity info from BigQuery ListenBrainz DB

## 4️⃣ Google BigQuery

In [4]:
!pip install --quiet pandas-gbq

In [5]:
import pandas_gbq

In [6]:
query = """
    SELECT artist_name, COUNT(artist_name) artist_count
    FROM `lwt-data-lectures.listenbrainz.listen` 
    WHERE listened_at BETWEEN '2017-01-01' AND '2018-01-01'
    GROUP BY artist_name
    HAVING artist_count > 1000
    ORDER BY artist_count DESC
    LIMIT 1000
"""

In [7]:
project_id='lwt-data-lectures'

In [8]:
musicbrainz_df = pandas_gbq.read_gbq(query, project_id=project_id)
musicbrainz_df

GenericGBQException: Reason: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/lwt-data-lectures/jobs?prettyPrint=false: Access Denied: Project lwt-data-lectures: User does not have bigquery.jobs.create permission in project lwt-data-lectures.

(job ID: b2f6dd93-dbf6-4dff-9dc4-b65ac077afd9)

                -----Query Job SQL Follows-----                 

    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:    SELECT artist_name, COUNT(artist_name) artist_count
   3:    FROM `lwt-data-lectures.listenbrainz.listen` 
   4:    WHERE listened_at BETWEEN '2017-01-01' AND '2018-01-01'
   5:    GROUP BY artist_name
   6:    HAVING artist_count > 1000
   7:    ORDER BY artist_count DESC
   8:    LIMIT 1000
    |    .    |    .    |    .    |    .    |    .    |

**Last Goal:** get the artist bday from Wikipedia!

## 5️⃣ Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def bday_scrape(name):
    try:
        url = f"https://en.wikipedia.org/wiki/{name.replace(' ', '_')}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.find("span", class_='bday').text
    except:
        return 'N/A'

In [ ]:
# for index, row in tracks_df.iterrows():
#     bday = bday_scrape(row['artists'])
#     tracks_df.loc[index, 'bday'] = bday
    
tracks_df['bday'] = tracks_df['artists'].apply(bday_scrape)
            

In [ ]:
tracks_df.head(10)

In [ ]:
tracks_df["listeners_lastfm"].hist(bins=20)

In [ ]:
import re
pattern = r'\d{4}-\d{2}-\d{2}'
# Find only the rows that have valid birthdays
only_bdays = tracks_df[tracks_df["bday"].str.match(pattern)].copy()
# Convert to a datetime format
tracks_df["birthday"] = pd.to_datetime(only_bdays["bday"])

In [ ]:
tracks_df.plot.scatter("birthday", "listeners_lastfm")